# E17: Characterizing Distributions with Moments and MGFs

**Objective:** This notebook provides a deep dive into the fundamental properties of several key probability distributions. We will move beyond simple mean and variance to calculate higher-order moments (skewness, kurtosis) and explore the powerful Moment Generating Function (MGF).

This exercise serves as a bridge between descriptive statistics and the core theoretical machinery of probability distributions. We will:
1.  **Calculate Foundational Moments:** For five different distributions, compute the first moment about the origin ($\mu'_1$, the mean), the second moment about the origin ($\mu'_2$), and the second central moment ($\mu_2$, the variance).
2.  **Analyze Distribution Shape:** Compute the standardized third and fourth moments to quantify skewness and kurtosis, providing a numerical description of each distribution's shape.
3.  **Leverage the MGF:** For the first four distributions, we will work with their Moment Generating Functions to verify how they can be used to derive the mean and variance.

### 1. Setup

We import `scipy.stats`, our primary tool for accessing the properties of these distributions. We then define the parameters for each case as specified in `Table 11`.

In [9]:
import numpy as np
from scipy.stats import bernoulli, binom, norm, chi2, t
import sympy as sp

# Define parameters for each case from Table 11
params = {
    'A': {'dist': bernoulli, 'p': 0.8},
    'B': {'dist': binom, 'n': 20, 'p': 0.3},
    'C': {'dist': norm, 'loc': 20, 'scale': 2}, # scale is std. dev., so sqrt(4)
    'D': {'dist': chi2, 'df': 20},
    'E': {'dist': t, 'df': 20}
}

### (a) Moments about the Origin and the Mean

For each distribution, we will compute:
-   $\mu'_1 = E[X]$ (Mean)
-   $\mu_2 = E[(X-E[X])^2]$ (Variance)
-   $\mu'_2 = E[X^2]$

We will use the convenient `stats()` method from `scipy`, which can provide these values directly. We will also use the fundamental relationship $\mu'_2 = \text{Var}(X) + (E[X])^2 = \mu_2 + (\mu'_1)^2$ to calculate the second moment about theorigin.

In [10]:
print("(a) Origin and Central Moments ---")

for case_id, p in params.items():
    dist_name = p['dist'].name
    # Unpack parameters for the specific distribution
    args = {k: v for k, v in p.items() if k != 'dist'}
    
    # Get mean and variance
    mean, var = p['dist'].stats(**args, moments='mv')
    
    # Calculate mu'_2 using the relation: var = E[X^2] - (E[X])^2
    mu_1_prime = float(mean)
    mu_2 = float(var)
    mu_2_prime = mu_2 + mu_1_prime**2
    
    print(f"\nCase {case_id} ({dist_name.capitalize()}):")
    print(f"  μ'_1 (Mean):              {mu_1_prime:.4f}")
    print(f"  μ_2 (Variance):           {mu_2:.4f}")
    print(f"  μ'_2 (Second Origin Mom): {mu_2_prime:.4f}")

(a) Origin and Central Moments ---

Case A (Bernoulli):
  μ'_1 (Mean):              0.8000
  μ_2 (Variance):           0.1600
  μ'_2 (Second Origin Mom): 0.8000

Case B (Binom):
  μ'_1 (Mean):              6.0000
  μ_2 (Variance):           4.2000
  μ'_2 (Second Origin Mom): 40.2000

Case C (Norm):
  μ'_1 (Mean):              20.0000
  μ_2 (Variance):           4.0000
  μ'_2 (Second Origin Mom): 404.0000

Case D (Chi2):
  μ'_1 (Mean):              20.0000
  μ_2 (Variance):           40.0000
  μ'_2 (Second Origin Mom): 440.0000

Case E (T):
  μ'_1 (Mean):              0.0000
  μ_2 (Variance):           1.1111
  μ'_2 (Second Origin Mom): 1.1111


### (b) Standardized Moments (Skewness and Kurtosis)

Now we analyze the shape. We'll compute:
-   **Skewness ($\beta_3$)**: A measure of asymmetry.
-   **Kurtosis ($\beta_4$)**: A measure of the "tailedness" relative to a Normal distribution. We will report the **excess kurtosis**, which is $\beta_4 - 3$.

For the Student's t-distribution, moments are only defined if the degrees of freedom (`df`) are greater than the moment order. For Case E (`df=20`), skewness (`moment=3`) and kurtosis (`moment=4`) are well-defined.

In [11]:
print("(b) Standardized Moments (Shape) ---")

for case_id, p in params.items():
    dist_name = p['dist'].name
    args = {k: v for k, v in p.items() if k != 'dist'}
    
    # Get skewness and kurtosis
    # The 's' and 'k' moments give skew and *excess* kurtosis
    _, _, skew, kurt = p['dist'].stats(**args, moments='mvsk')
    
    print(f"\nCase {case_id} ({dist_name.capitalize()}):")
    
    if np.isnan(skew):
        print(f"  β_3 (Skewness):         Does not exist")
    else:
        print(f"  β_3 (Skewness):         {float(skew):.4f}")
        
    if np.isnan(kurt):
         print(f"  Excess Kurtosis:        Does not exist")
    else:
        print(f"  Excess Kurtosis:        {float(kurt):.4f}")

(b) Standardized Moments (Shape) ---

Case A (Bernoulli):
  β_3 (Skewness):         -1.5000
  Excess Kurtosis:        0.2500

Case B (Binom):
  β_3 (Skewness):         0.1952
  Excess Kurtosis:        -0.0619

Case C (Norm):
  β_3 (Skewness):         0.0000
  Excess Kurtosis:        0.0000

Case D (Chi2):
  β_3 (Skewness):         0.6325
  Excess Kurtosis:        0.6000

Case E (T):
  β_3 (Skewness):         0.0000
  Excess Kurtosis:        0.3750


### (c) Moment Generating Functions (MGF)

Finally, we'll demonstrate the power of the MGF for the first four cases. We will:
1.  Define the MGF for each distribution symbolically using `sympy`.
2.  Calculate its first and second derivatives with respect to `t`.
3.  Evaluate these derivatives at `t=0` to find $\mu'_1$ and $\mu'_2$.
4.  Verify that these results match our calculations from Part (a).

**Note:** The MGF for the Student's t-distribution is not defined, so we exclude Case E.

In [12]:
import sympy as sp

# Define t as a symbolic variable
t = sp.Symbol('t')

# --- MGF Definitions ---
p_A = params['A']['p']
n_B, p_B = params['B']['n'], params['B']['p']
mu_C, sigma_C = params['C']['loc'], params['C']['scale']
df_D = params['D']['df']

mgfs = {
    'A': (1 - p_A) + p_A * sp.exp(t),
    'B': ( (1 - p_B) + p_B * sp.exp(t) )**n_B,
    'C': sp.exp(mu_C * t + (sigma_C**2 * t**2) / 2),
    'D': (1 - 2*t)**(-df_D / 2)
}

print("--- Part (c): Verification using MGFs ---")

for case_id, mgf in mgfs.items():
    dist_name = params[case_id]['dist'].name
    
    # Calculate derivatives
    mgf_prime_1 = sp.diff(mgf, t)
    mgf_prime_2 = sp.diff(mgf_prime_1, t)
    
    # Evaluate at t=0
    mu_1_prime_mgf = mgf_prime_1.evalf(subs={t: 0})
    mu_2_prime_mgf = mgf_prime_2.evalf(subs={t: 0})
    
    # Calculate variance from MGF moments
    var_mgf = mu_2_prime_mgf - mu_1_prime_mgf**2
    
    # Get known values from part (a) for comparison
    mean_known, var_known = params[case_id]['dist'].stats(**{k: v for k, v in params[case_id].items() if k != 'dist'}, moments='mv')

    print(f"\nCase {case_id} ({dist_name.capitalize()}):")
    print(f"  MGF: {mgf}")
    print(f"  E[X] from MGF M'(0):   {mu_1_prime_mgf:.4f} (Known: {float(mean_known):.4f})")
    print(f"  Var(X) from MGF:       {var_mgf:.4f} (Known: {float(var_known):.4f})")
    
    assert np.isclose(float(mu_1_prime_mgf), float(mean_known))
    assert np.isclose(float(var_mgf), float(var_known))
    print("  ✅ Verification successful.")

--- Part (c): Verification using MGFs ---

Case A (Bernoulli):
  MGF: 0.8*exp(t) + 0.2
  E[X] from MGF M'(0):   0.8000 (Known: 0.8000)
  Var(X) from MGF:       0.1600 (Known: 0.1600)
  ✅ Verification successful.

Case B (Binom):
  MGF: (0.3*exp(t) + 0.7)**20
  E[X] from MGF M'(0):   6.0000 (Known: 6.0000)
  Var(X) from MGF:       4.2000 (Known: 4.2000)
  ✅ Verification successful.

Case C (Norm):
  MGF: exp(2*t**2 + 20*t)
  E[X] from MGF M'(0):   20.0000 (Known: 20.0000)
  Var(X) from MGF:       4.0000 (Known: 4.0000)
  ✅ Verification successful.

Case D (Chi2):
  MGF: (1 - 2*t)**(-10.0)
  E[X] from MGF M'(0):   20.0000 (Known: 20.0000)
  Var(X) from MGF:       40.0000 (Known: 40.0000)
  ✅ Verification successful.
